<a href="https://colab.research.google.com/github/elenadanchenkomn/data_analysis/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%A2%D1%80%D0%B5%D1%82%D0%B8%D0%B9_%D0%BA%D0%B5%D0%B9%D1%81_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3 кейс

**В этом кейсе вы будете рассчитывать:**
* retention
* rolling retention
* lifetime
* churn rate
* mau
* wau
* dau

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [5]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2025-11-17 06:22:47--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 140.82.113.4
Connecting to gist.github.com (gist.github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2025-11-17 06:22:47--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: ‘registrations.csv.1’

registrations.csv.1 100%[===================>]  14.57K  --.-KB/s    in 0s      

2025-11-17 06:22:47 (127 

Файлами для работы являются `registrations.csv` и `entries.csv`. В них хранятся данные о регистрациях пользователей и входа на платформу соответственно.

### **Посчитайте Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе**

Cохраните результат в переменную `retention_15_day`

**Примечание:** результат округлите до 5 знаков после запятой

In [17]:
# Ваше решение
import pandas as pd
registrations = pd.read_csv('registrations.csv',sep=";")
entries = pd.read_csv('entries.csv' ,sep=";" )
registrations["registration_date"] = pd.to_datetime(registrations["registration_date"])
entries["entry_date"] = pd.to_datetime(entries["entry_date"])
jan_regs = registrations[registrations["registration_date"].dt.month == 1]
jan_entries = entries.merge(jan_regs[["user_id", "registration_date"]], on="user_id", how="inner")
jan_entries["days_after_reg"] = (jan_entries["entry_date"] - jan_entries["registration_date"]).dt.days
returned_day_15 = jan_entries[jan_entries["days_after_reg"] == 15]["user_id"].nunique()
total_jan_users = jan_regs["user_id"].nunique()
retention_15_day = round(returned_day_15 / total_jan_users * 100, 5)
retention_15_day

54.65116

In [18]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
# Открываем файл с правильными ответами
with open('metrics.txt', 'r') as f:
    answers = f.read().split('\n')

correct_answer = float(answers[0])

try:
    assert retention_15_day == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Rolling-retention 30 дня (в процентах) для пользователей из той же когорты**

Сохраните результат в переменную `rolling_retention`

**Примечание:** результат округлите до 5 знаков после запятой

In [23]:
# Ваше решение
returned_day_30 = jan_entries[jan_entries["days_after_reg"] >= 30]["user_id"].nunique()
total_jan_users = jan_regs["user_id"].nunique()
rolling_retention = round(returned_day_30 / total_jan_users * 100, 5)
rolling_retention

29.06977

In [24]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[1])

try:
    assert rolling_retention == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Lifetime по всем пользователям, посчитанный как интеграл от n-day retention**

Сохраните результат в переменную `lifetime`

**Примечание:** результат округлите до 5 знаков после запятой

In [30]:
# Ваше решение
all_entries = entries.merge(registrations[["user_id", "registration_date"]], on="user_id", how="inner")
all_entries["days_after_reg"] = (all_entries["entry_date"] - all_entries["registration_date"]).dt.days
retention_by_day = all_entries.groupby("days_after_reg")["user_id"].nunique()
total_users = registrations["user_id"].nunique()
retention_by_day_fraction = retention_by_day / total_users
lifetime = round(retention_by_day_fraction.sum(), 5)
lifetime

np.float64(14.804)

In [31]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[2])

try:
    assert lifetime == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Churn rate 29 дня (в долях), посчитанный по всем пользователям**

Сохраните результат в переменную `churn_29`.  
Если будете считать CR от Retention, ведите расчет от Rolling Retention, таким образом, мы получим всех, кто не заходил в 29 день и после. Ведя расчет от обычного Retention, наоборот - получим только CR в 29 день.


In [33]:
# Ваше решение
rolling_retention_29 = (all_entries[all_entries["days_after_reg"] >= 29]["user_id"].nunique()) / total_users
churn_29 = 1 - rolling_retention_29
churn_29

0.509

In [34]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[3])

try:
    assert churn_29 == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau за последний месяц/неделю/день записей**

Сохраните результат в переменные `dec_mau`, `dec_wau`, `dec_dau` соответственно

**Примечание:** последний месяц записей - декабрь. Поэтому `mau` рассчитываем для декабря (2021 года), для `wau` берем последнюю неделю - с 25 по 31 декабря, и для `dau` соответственно последний день - 31 декабря.

In [35]:
# Ваше решение
dec_mau = entries[
    (entries["entry_date"].dt.year == 2021) &
    (entries["entry_date"].dt.month == 12)
]["user_id"].nunique()

dec_mau

133

In [38]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[4])

try:
    assert dec_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [36]:
# Ваше решение
start_week = pd.Timestamp("2021-12-25")
end_week = pd.Timestamp("2021-12-31")

dec_wau = entries[
    (entries["entry_date"] >= start_week) &
    (entries["entry_date"] <= end_week)
]["user_id"].nunique()

dec_wau

84

In [37]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[5])

try:
    assert dec_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [39]:
# Ваше решение
last_day = pd.Timestamp("2021-12-31")

dec_dau = entries[
    entries["entry_date"] == last_day
]["user_id"].nunique()

dec_mau, dec_wau, dec_dau

dec_dau

47

In [40]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[6])

try:
    assert dec_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


### **Посчитайте Mau, Wau, Dau усредненные**

Сохраните результат в переменные `avg_mau`, `avg_wau`, `avg_dau` соответственно

**Примечание:** результаты округлите до 5 знаков после запятой

In [42]:
# Ваше решение

df = entries.copy()

df["year"] = df["entry_date"].dt.year
df["month"] = df["entry_date"].dt.month
df["week"] = df["entry_date"].dt.isocalendar().week
df["day"] = df["entry_date"].dt.date
mau_values = df.groupby(["year", "month"])["user_id"].nunique()
avg_mau = round(mau_values.mean(), 5)
avg_mau

np.float64(102.58333)

In [43]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[7])

try:
    assert avg_mau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [44]:
# Ваше решение

wau_values = df.groupby(["year", "week"])["user_id"].nunique()
avg_wau = round(wau_values.mean(), 5)
avg_wau

np.float64(89.86792)

In [45]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[8])

try:
    assert avg_wau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


In [46]:
# Ваше решение

dau_values = df.groupby("day")["user_id"].nunique()
avg_dau = round(dau_values.mean(), 5)
avg_dau

np.float64(40.5589)

In [47]:
#@title ✏️ Проверка: чтобы проверить свое решение запустите код в этой ячейке
correct_answer = float(answers[9])

try:
    assert avg_dau == correct_answer
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
